# A Marlowe Runtime Client for AWS Lambda


**Executive Summary**

We introduce a Marlowe Runtime client for the AWS Lambda service using the [marlowe-lambda](https://github.com/input-output-hk/marlowe-lambda) Haskell package. This permits users to create, apply inputs to, and withdraw funds from Marlowe contract instances; it also lets them list all of the Marlowe contracts on the blockchain and to examine their on-chain status and contents.  Users do not need to install any Marlowe software and the only Cardano software they need is a wallet for signing the transactions. (The example here uses several command-line tools for convenience, but any toolset for calling AWS Lambda, manipulating JSON, and signing/submitting transactions could be used.) AWS services are monetizable, via a charge for contract creation and/or staking rewards that accrue to the publisher of the AWS Lambda service. Although, we demonstrate the basic use case with a simple zero-coupon bond (also known as a "bullet loan") where a lender provide funds to a borrower who repays both principal and interest, the service may be used with any Marlowe contract. In addition to providing the convenience of zero-installation use of Marlowe, AWS Lambda also supports the orchestration of complex workflows involving multiple Marlowe contracts or a mixture of Marlowe and non-Marlowe lambdas.

A video demonstration of this notebook is available [here](https://youtu.be/huXbRyrmW60).


![Zero-coupon bond contract in Blockly](zcb.png)

## Preliminaries

### Record version numbers for future reference

In [1]:
aws --version

aws-cli/2.4.23 Python/3.9.11 Linux/5.10.126 source/x86_64.nixos.22 prompt/off


In [2]:
cardano-cli --version

cardano-cli 1.35.3 - linux-x86_64 - ghc-8.10
git rev 0000000000000000000000000000000000000000


In [3]:
marlowe-cli --version

marlowe-cli 0.0.10.0


### Set the network for local operations such as signing transactions

The AWS Lambda operation use their own node, but we will be locally signing and submitting the transactions it builds. This could be done by a wallet.

In [4]:
export CARDANO_NODE_SOCKET_PATH=node.socket
export CARDANO_TESTNET_MAGIC=2
MAGIC=(--testnet-magic 2)

### Setup the addresses and keys for the parties to the contract

In [5]:
export TREASURY=treasury

#### The lender

In [6]:
LENDER_SKEY="$TREASURY/john-fletcher.skey"
LENDER_ADDR="$(cat $TREASURY/john-fletcher.testnet.address)"
echo "LENDER_ADDR = $LENDER_ADDR"

LENDER_ADDR = addr_test1vqwt2xlr4d8yk4qws675exlqy6pdhq2s76wrehkjggkvr0cerfe8r


See that the lender has funds.

In [7]:
cardano-cli query utxo "${MAGIC[@]}" --address "$LENDER_ADDR"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
dc8fceb2f336dedf2af52ee3eb1cc469f849bf3c277919d665beb0fbeecc1ec6     2        150000000000 lovelace + TxOutDatumNone


#### The borrower

In [8]:
BORROWER_SKEY="$TREASURY/thomas-kyd.skey"
BORROWER_ADDR="$(cat $TREASURY/thomas-kyd.testnet.address)"
echo "BORROWER_ADDR = $BORROWER_ADDR"

BORROWER_ADDR = addr_test1vr7n0zzth5zycuh972w7rdmh48qur4f3wu6ntn2m2h30dlcvltuy5


See that the borrower has funds.

In [9]:
cardano-cli query utxo "${MAGIC[@]}" --address "$BORROWER_ADDR"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
dc8fceb2f336dedf2af52ee3eb1cc469f849bf3c277919d665beb0fbeecc1ec6     3        150000000000 lovelace + TxOutDatumNone


### The AWS Lambda function for Marlowe Runtime

![Marlowe Runtime](marlowe-runtime.png)

The process for deploying the Marlowe Runtime client for AWS Lambda is straightforward.

First clone the respository and build the software.

```bash
git clone https://github.com/input-output-hk/marlowe-lambda.git
cd marlowe-lambda
nix develop
./package.sh
```
Then upload the resulting `marlowe-lambda.zip` file to AWS Lambda.

The lambda client requires a connection to the Marlowe Runtime backend, which can be deployed as a Docker image. Configuration files are provided for connecting to the `preprod` and `preview` testnets, or to `mainnet`.

Check the that AWS Lambda function for Marlowe Runtime has been registered.

In [10]:
aws lambda list-functions | json2yaml

Functions:
- Architectures:
  - x86_64
  CodeSha256: FgdMYYVc6N40litNVP7xc0aZh2oS4BA159Rn/+oInVM=
  CodeSize: 23071139
  Description: ''
  FunctionArn: arn:aws:lambda:us-west-2:454236594309:function:bbush-marlowe-runtime-1
  FunctionName: bbush-marlowe-runtime-1
  Handler: marlowe
  LastModified: 2022-10-31T20:47:13.000+0000
  MemorySize: 128
  PackageType: Zip
  RevisionId: e7ca2663-bd47-4cd0-872c-f40726f738f7
  Role: arn:aws:iam::454236594309:role/service-role/bbush-marlowe-runtime-1-role-4i77polb
  Runtime: provided
  Timeout: 25
  TracingConfig:
    Mode: PassThrough
  Version: $LATEST


## The contract

Set parameters for the contract.

In [11]:
SECOND=1000
MINUTE=$((60 * SECOND))
HOUR=$((60 * MINUTE))

NOW=$(($(date -u +%s) * SECOND))

ADA=1000000
THOUSAND=1000
MINIMUM_ADA=$((3 * ADA))

PRINCIPAL=$((100 * THOUSAND * ADA))
INTEREST=$((5 * THOUSAND * ADA))

LENDING_DEADLINE=$((NOW + 2 * HOUR))
REPAYMENT_DEADLINE=$((NOW + 6 * HOUR))

We can create the contract with [Marlowe Playground](https://marlowe-playground-staging.plutus.aws.iohkdev.io/) or with any tool that can write JSON in the correct format. For convenience, however, we use `marlowe-cli` to generate the zero-coupon bond contract from a template.

In [12]:
marlowe-cli template zcb \
  --minimum-ada "$MINIMUM_ADA" \
  --lender "$LENDER_ADDR" \
  --borrower "$BORROWER_ADDR" \
  --principal "$PRINCIPAL" \
  --interest "$INTEREST" \
  --lending-deadline "$LENDING_DEADLINE" \
  --repayment-deadline "$REPAYMENT_DEADLINE" \
  --out-contract-file zcb-1.contract \
  --out-state-file /dev/null

## Transaction 1. Create the contract

We splice the contract into a request that will be send to the lambda.

In [13]:
cat << EOI > zcb-1.request
{                                                                                                                                    
  "request" : "create",                
  "contract" : $(cat zcb-1.contract),
  "minUtxo" : $MINIMUM_ADA,              
  "roles" : {},                             
  "change" : "$LENDER_ADDR",
  "addresses" : [],                                         
  "collateral" : []                                  
}
EOI

json2yaml zcb-1.request

addresses: []
change: addr_test1vqwt2xlr4d8yk4qws675exlqy6pdhq2s76wrehkjggkvr0cerfe8r
collateral: []
contract:
  timeout: 1667257669000
  timeout_continuation: close
  when:
  - case:
      deposits: 100000000000
      into_account:
        address: addr_test1vqwt2xlr4d8yk4qws675exlqy6pdhq2s76wrehkjggkvr0cerfe8r
      of_token:
        currency_symbol: ''
        token_name: ''
      party:
        address: addr_test1vqwt2xlr4d8yk4qws675exlqy6pdhq2s76wrehkjggkvr0cerfe8r
    then:
      from_account:
        address: addr_test1vqwt2xlr4d8yk4qws675exlqy6pdhq2s76wrehkjggkvr0cerfe8r
      pay: 100000000000
      then:
        timeout: 1667272069000
        timeout_continuation: close
        when:
        - case:
            deposits:
              add: 100000000000
              and: 5000000000
            into_account:
              address: addr_test1vr7n0zzth5zycuh972w7rdmh48qur4f3wu6ntn2m2h30dlcvltuy5
            of_token:
              currency_symbol: ''
              token_name: ''

Call the lambda to create the transaction.

In [14]:
aws lambda invoke \
  --function-name arn:aws:lambda:us-west-2:454236594309:function:bbush-marlowe-runtime-1 \
  --cli-binary-format raw-in-base64-out \
  --payload file://zcb-1.request \
  zcb-1.response

{
    "StatusCode": 200,
    "ExecutedVersion": "$LATEST"
}


The response contains the transaction that needs to be signed by the wallet.

In [15]:
json2yaml zcb-1.response

body:
  cborHex: 86a50081825820dc8fceb2f336dedf2af52ee3eb1cc469f849bf3c277919d665beb0fbeecc1ec6020d81825820dc8fceb2f336dedf2af52ee3eb1cc469f849bf3c277919d665beb0fbeecc1ec6020182a200581d601cb51be3ab4e4b540e86bd4c9be02682db8150f69c3cded2422cc1bf011b00000022ec81730ba300581d706a9391d6aa51af28dd876ebb5565b69d1e83e5ac7861506bd29b56b0011a002dc6c00282005820f655774fa192c7c4452d1cf969ffcea6f28c0826928c270c78e5eca37ae2b20d021a000322350b58208658ad1a032d64874b6944853a2c95f62b58f8a6bb0b93a5e9f66bc706ccb4779fff81d8799fd8799f40ffd8799fa1d8799fd8799fd87980d8799fd8799f581c1cb51be3ab4e4b540e86bd4c9be02682db8150f69c3cded2422cc1bfffd87a80ffffd8799f4040ffff1a002dc6c0a0a000ffd87c9f9fd8799fd8799fd8799fd87980d8799fd8799f581c1cb51be3ab4e4b540e86bd4c9be02682db8150f69c3cded2422cc1bfffd87a80ffffd8799fd87980d8799fd8799f581c1cb51be3ab4e4b540e86bd4c9be02682db8150f69c3cded2422cc1bfffd87a80ffffd8799f4040ffd87a9f1b000000174876e800ffffd87a9fd8799fd87980d8799fd8799f581c1cb51be3ab4e4b540e86bd4c9be02682db8150f69c3cded2422cc

Marlowe Runtime does not manage or access signing keys. We could use a CIP-30 light wallet (such as Lace), `cardano-wallet`, `cardano-hw-cli`, or `cardano-cli`. For convenience, we use `marlowe-cli` because it can sign, submit, and wait for confirmation.

In [16]:
jq '.body' zcb-1.response > zcb-1.txBody

marlowe-cli transaction submit \
  --tx-body-file zcb-1.txBody \
  --required-signer "$LENDER_SKEY" \
  --timeout 600

TxId "66e4af5ec99f09c28c6378e1d9d9508d95544e258fd8c8e7f3ff168a73e7b656"


Each Marlowe contract is identified by the UTxO of its creation transaction.

In [17]:
CONTRACT_ID="$(jq -r '.contractId' zcb-1.response)"
echo "CONTRACT_ID = $CONTRACT_ID"

CONTRACT_ID = 66e4af5ec99f09c28c6378e1d9d9508d95544e258fd8c8e7f3ff168a73e7b656#1


Now query the blockchain to see that that contract has been created. We could use any Cardano explorer for this, but here we use `cardano-cli`.

In [18]:
cardano-cli query utxo "${MAGIC[@]}" --tx-in "$CONTRACT_ID"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
66e4af5ec99f09c28c6378e1d9d9508d95544e258fd8c8e7f3ff168a73e7b656     1        3000000 lovelace + TxOutDatumHash ScriptDataInBabbageEra "f655774fa192c7c4452d1cf969ffcea6f28c0826928c270c78e5eca37ae2b20d"


## Retrieve the on-chain information about the contract

Now get information about the contract using the lambda. First we construct a request.

In [19]:
yaml2json << EOI > zcb-get.request
request: get
contractId: $CONTRACT_ID
EOI

cat zcb-get.request

{"contractId":"66e4af5ec99f09c28c6378e1d9d9508d95544e258fd8c8e7f3ff168a73e7b656#1","request":"get"}


Now call the lambda.

In [20]:
aws lambda invoke \
  --function-name arn:aws:lambda:us-west-2:454236594309:function:bbush-marlowe-runtime-1 \
  --cli-binary-format raw-in-base64-out \
  --payload file://zcb-get.request \
  zcb-get.response

{
    "StatusCode": 200,
    "ExecutedVersion": "$LATEST"
}


View the resulting complete information about the contract and its creation.

In [21]:
json2yaml zcb-get.response

creation:
  output:
    address: 706a9391d6aa51af28dd876ebb5565b69d1e83e5ac7861506bd29b56b0
    assets:
      ada: 3000000
      tokens: []
    datum:
      marloweContract:
        timeout: 1667257669000
        timeout_continuation: close
        when:
        - case:
            deposits: 100000000000
            into_account:
              address: addr_test1vqwt2xlr4d8yk4qws675exlqy6pdhq2s76wrehkjggkvr0cerfe8r
            of_token:
              currency_symbol: ''
              token_name: ''
            party:
              address: addr_test1vqwt2xlr4d8yk4qws675exlqy6pdhq2s76wrehkjggkvr0cerfe8r
          then:
            from_account:
              address: addr_test1vqwt2xlr4d8yk4qws675exlqy6pdhq2s76wrehkjggkvr0cerfe8r
            pay: 100000000000
            then:
              timeout: 1667272069000
              timeout_continuation: close
              when:
              - case:
                  deposits:
                    add: 100000000000
                    and: 5

## Transaction 2. Deposit the loan amount

The lender deposits the principal. Once again, we use `marlowe-cli` to prepare the JSON, but this could be done by any JSON tooling that outputs properly-formatted Marlowe input.

In [22]:
marlowe-cli input deposit \
  --deposit-party "$LENDER_ADDR" \
  --deposit-account "$LENDER_ADDR" \
  --deposit-amount "$PRINCIPAL" \
  --out-file zcb-2.input

Now bundle this information into a request that contains additional information about the transaction to be created.

In [23]:
cat << EOI > zcb-2.request
{
  "request" : "apply",
  "contractId" : "$CONTRACT_ID",
  "inputs" : [
    $(cat zcb-2.input)
  ],
  "validityLowerBound" : $(($(date -u +%s) * SECOND - 1 * MINUTE)),
  "validityUpperBound" : $(($(date -u +%s) * SECOND + 2 * MINUTE)),
  "change" : "$LENDER_ADDR",
  "addresses" : [],
  "collateral" : []
}
EOI

json2yaml zcb-2.request

addresses: []
change: addr_test1vqwt2xlr4d8yk4qws675exlqy6pdhq2s76wrehkjggkvr0cerfe8r
collateral: []
contractId: 66e4af5ec99f09c28c6378e1d9d9508d95544e258fd8c8e7f3ff168a73e7b656#1
inputs:
- input_from_party:
    address: addr_test1vqwt2xlr4d8yk4qws675exlqy6pdhq2s76wrehkjggkvr0cerfe8r
  into_account:
    address: addr_test1vqwt2xlr4d8yk4qws675exlqy6pdhq2s76wrehkjggkvr0cerfe8r
  of_token:
    currency_symbol: ''
    token_name: ''
  that_deposits: 100000000000
request: apply
validityLowerBound: 1667250780000
validityUpperBound: 1667250960000


Now call the lambda function for Marlowe Runtime.

In [24]:
aws lambda invoke \
  --function-name arn:aws:lambda:us-west-2:454236594309:function:bbush-marlowe-runtime-1 \
  --cli-binary-format raw-in-base64-out \
  --payload file://zcb-2.request \
  zcb-2.response

{
    "StatusCode": 200,
    "ExecutedVersion": "$LATEST"
}


Check that the response created a transaction body.

In [25]:
json2yaml zcb-2.response

body:
  cborHex: 86a9008282582066e4af5ec99f09c28c6378e1d9d9508d95544e258fd8c8e7f3ff168a73e7b6560082582066e4af5ec99f09c28c6378e1d9d9508d95544e258fd8c8e7f3ff168a73e7b656010d8182582066e4af5ec99f09c28c6378e1d9d9508d95544e258fd8c8e7f3ff168a73e7b656001281825820087f21f109a997193421a81886ea8c6397d336d19e696457b9c5c7aefdc31873010183a200581d601cb51be3ab4e4b540e86bd4c9be02682db8150f69c3cded2422cc1bf011b0000000ba3ffce4da300581d706a9391d6aa51af28dd876ebb5565b69d1e83e5ac7861506bd29b56b0011a002dc6c002820058201191de86231ce0a59716ea1e3dfd402e7811d0176fc18b2e38763bb3aa0aa34ea200581d60fd37884bbd044c72e5f29de1b777a9c1c1d531773535cd5b55e2f6ff011b000000174876e800021a000abcbe031a006e9790081a006e96dc0e81581c1cb51be3ab4e4b540e86bd4c9be02682db8150f69c3cded2422cc1bf0b5820360e8170c5dc53fb8f16be048b974f12a393127bce32c553aff896b56d7a8fff9fff82d8799fd8799f40ffd8799fa1d8799fd8799fd87980d8799fd8799f581c1cb51be3ab4e4b540e86bd4c9be02682db8150f69c3cded2422cc1bfffd87a80ffffd8799f4040ffff1a002dc6c0a0a01b000001842fe3f760ffd

Now sign and submit the transaction locally.

In [26]:
jq '.body' zcb-2.response > zcb-2.txBody

marlowe-cli transaction submit \
  --tx-body-file zcb-2.txBody \
  --required-signer "$LENDER_SKEY" \
  --timeout 600

TxId "562ef5fae1a44493a63dac0c6cc2582859b0f19f39b4875350a3e93a4b8b7495"


Marlowe contracts reside at a well-known universal address, which we calculate. We could also look this up in a Cardano explorer.

In [27]:
CONTRACT_ADDR="$(marlowe-cli contract address)"
echo "CONTRACT_ADDR = $CONTRACT_ADDR"

CONTRACT_ADDR = addr_test1wp4f8ywk4fg672xasahtk4t9k6w3aql943uxz5rt62d4dvqu3c6jv


Now query the blockchain at the contract address to see the UTxO that was created there.

In [28]:
TX_2="$(jq -r '.txId' zcb-2.response)"

cardano-cli query utxo "${MAGIC[@]}" --address "$CONTRACT_ADDR" | sed -n -e "1,2p;/$TX_2/p"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
562ef5fae1a44493a63dac0c6cc2582859b0f19f39b4875350a3e93a4b8b7495     1        3000000 lovelace + TxOutDatumHash ScriptDataInBabbageEra "1191de86231ce0a59716ea1e3dfd402e7811d0176fc18b2e38763bb3aa0aa34e"


## Retrieve the on-chain information about the contract

We now retrieve the latest on-chain information about the contract by calling the lambda again.

In [29]:
aws lambda invoke \
  --function-name arn:aws:lambda:us-west-2:454236594309:function:bbush-marlowe-runtime-1 \
  --cli-binary-format raw-in-base64-out \
  --payload file://zcb-get.request \
  zcb-get.response

{
    "StatusCode": 200,
    "ExecutedVersion": "$LATEST"
}


View the resulting information about the contract, which shows both the creation transaction and the first step that deposited the principal.

In [30]:
json2yaml zcb-get.response

creation:
  output:
    address: 706a9391d6aa51af28dd876ebb5565b69d1e83e5ac7861506bd29b56b0
    assets:
      ada: 3000000
      tokens: []
    datum:
      marloweContract:
        timeout: 1667257669000
        timeout_continuation: close
        when:
        - case:
            deposits: 100000000000
            into_account:
              address: addr_test1vqwt2xlr4d8yk4qws675exlqy6pdhq2s76wrehkjggkvr0cerfe8r
            of_token:
              currency_symbol: ''
              token_name: ''
            party:
              address: addr_test1vqwt2xlr4d8yk4qws675exlqy6pdhq2s76wrehkjggkvr0cerfe8r
          then:
            from_account:
              address: addr_test1vqwt2xlr4d8yk4qws675exlqy6pdhq2s76wrehkjggkvr0cerfe8r
            pay: 100000000000
            then:
              timeout: 1667272069000
              timeout_continuation: close
              when:
              - case:
                  deposits:
                    add: 100000000000
                    and: 5

## Transaction 3. The borrower deposits principal and interest

Since this is a "bullet loan", the borrower closes the contract by repaying the principal plus interest. Once again, we use `marlowe-cli` for convenience to create the JSON for the input.

In [31]:
marlowe-cli input deposit \
  --deposit-party "$BORROWER_ADDR" \
  --deposit-account "$BORROWER_ADDR" \
  --deposit-amount "$((PRINCIPAL + INTEREST))" \
  --out-file zcb-3.input

Similarly to the previous transaction, we splice the input into a request for the lambda.

In [32]:
cat << EOI > zcb-3.request
{
  "request" : "apply",
  "contractId" : "$CONTRACT_ID",
  "inputs" : [
    $(cat zcb-3.input)
  ],
  "validityLowerBound" : $(($(date -u +%s) * SECOND - 1 * MINUTE)),
  "validityUpperBound" : $(($(date -u +%s) * SECOND + 2 * MINUTE)),
  "change" : "$BORROWER_ADDR",
  "addresses" : [],
  "collateral" : []
}
EOI

json2yaml zcb-3.request

addresses: []
change: addr_test1vr7n0zzth5zycuh972w7rdmh48qur4f3wu6ntn2m2h30dlcvltuy5
collateral: []
contractId: 66e4af5ec99f09c28c6378e1d9d9508d95544e258fd8c8e7f3ff168a73e7b656#1
inputs:
- input_from_party:
    address: addr_test1vr7n0zzth5zycuh972w7rdmh48qur4f3wu6ntn2m2h30dlcvltuy5
  into_account:
    address: addr_test1vr7n0zzth5zycuh972w7rdmh48qur4f3wu6ntn2m2h30dlcvltuy5
  of_token:
    currency_symbol: ''
    token_name: ''
  that_deposits: 105000000000
request: apply
validityLowerBound: 1667250935000
validityUpperBound: 1667251115000


We now invoke the lambda to create the transaction.

In [33]:
aws lambda invoke \
  --function-name arn:aws:lambda:us-west-2:454236594309:function:bbush-marlowe-runtime-1 \
  --cli-binary-format raw-in-base64-out \
  --payload file://zcb-3.request \
  zcb-3.response

{
    "StatusCode": 200,
    "ExecutedVersion": "$LATEST"
}


Check the response to see that the transaction body was created.

In [34]:
json2yaml zcb-3.response

body:
  cborHex: 86a90082825820562ef5fae1a44493a63dac0c6cc2582859b0f19f39b4875350a3e93a4b8b749501825820dc8fceb2f336dedf2af52ee3eb1cc469f849bf3c277919d665beb0fbeecc1ec6030d81825820562ef5fae1a44493a63dac0c6cc2582859b0f19f39b4875350a3e93a4b8b7495021281825820087f21f109a997193421a81886ea8c6397d336d19e696457b9c5c7aefdc31873010182a200581d60fd37884bbd044c72e5f29de1b777a9c1c1d531773535cd5b55e2f6ff011b0000000a7a2cff3ba200581d601cb51be3ab4e4b540e86bd4c9be02682db8150f69c3cded2422cc1bf011b0000001872aaa0c0021a000882c5031a006e982b081a006e97770e81581cfd37884bbd044c72e5f29de1b777a9c1c1d531773535cd5b55e2f6ff0b582069e83989881721b447d7424d675c192b4c9083922bb9a989f3c09164cf89db6f9fff81d8799fd8799f40ffd8799fa1d8799fd8799fd87980d8799fd8799f581c1cb51be3ab4e4b540e86bd4c9be02682db8150f69c3cded2422cc1bfffd87a80ffffd8799f4040ffff1a002dc6c0a0a01b000001842fe3f760ffd87c9f9fd8799fd8799fd8799fd87980d8799fd8799f581cfd37884bbd044c72e5f29de1b777a9c1c1d531773535cd5b55e2f6ffffd87a80ffffd8799fd87980d8799fd8799f581cfd37884bb

Once again, we sign and submit the transaction locally, and wait for confirmation.

In [35]:
jq '.body' zcb-3.response > zcb-3.txBody

marlowe-cli transaction submit \
  --tx-body-file zcb-3.txBody \
  --required-signer "$BORROWER_SKEY" \
  --timeout 600

TxId "5aaa9332ee34f487122d1f8fe5619f930186512524b0c2bc85de9bdab30e07cc"


Because the contract has closed, we see that there is no UTxO at the contract address.

In [36]:
TX_3="$(jq -r '.txId' zcb-2.response)"

cardano-cli query utxo "${MAGIC[@]}" --address "$CONTRACT_ADDR" | sed -n -e "1,2p;/$TX_3/p"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------


The lender now has these funds in their wallet.

In [37]:
cardano-cli query utxo "${MAGIC[@]}" --address "$LENDER_ADDR"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
562ef5fae1a44493a63dac0c6cc2582859b0f19f39b4875350a3e93a4b8b7495     0        49996090957 lovelace + TxOutDatumNone
5aaa9332ee34f487122d1f8fe5619f930186512524b0c2bc85de9bdab30e07cc     1        105003000000 lovelace + TxOutDatumNone


The borrower now has these funds in their wallet.

In [38]:
cardano-cli query utxo "${MAGIC[@]}" --address "$BORROWER_ADDR"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
562ef5fae1a44493a63dac0c6cc2582859b0f19f39b4875350a3e93a4b8b7495     2        100000000000 lovelace + TxOutDatumNone
5aaa9332ee34f487122d1f8fe5619f930186512524b0c2bc85de9bdab30e07cc     0        44999442235 lovelace + TxOutDatumNone


## Retrieve the on-chain information about the contract

Fetch the latest on-chain information about the contract.

In [39]:
aws lambda invoke \
  --function-name arn:aws:lambda:us-west-2:454236594309:function:bbush-marlowe-runtime-1 \
  --cli-binary-format raw-in-base64-out \
  --payload file://zcb-get.request \
  zcb-get.response

{
    "StatusCode": 200,
    "ExecutedVersion": "$LATEST"
}


View the resulting information about the contract, seeing that the history now shows the final on-chain step that closed the contract.

In [40]:
json2yaml zcb-get.response

creation:
  output:
    address: 706a9391d6aa51af28dd876ebb5565b69d1e83e5ac7861506bd29b56b0
    assets:
      ada: 3000000
      tokens: []
    datum:
      marloweContract:
        timeout: 1667257669000
        timeout_continuation: close
        when:
        - case:
            deposits: 100000000000
            into_account:
              address: addr_test1vqwt2xlr4d8yk4qws675exlqy6pdhq2s76wrehkjggkvr0cerfe8r
            of_token:
              currency_symbol: ''
              token_name: ''
            party:
              address: addr_test1vqwt2xlr4d8yk4qws675exlqy6pdhq2s76wrehkjggkvr0cerfe8r
          then:
            from_account:
              address: addr_test1vqwt2xlr4d8yk4qws675exlqy6pdhq2s76wrehkjggkvr0cerfe8r
            pay: 100000000000
            then:
              timeout: 1667272069000
              timeout_continuation: close
              when:
              - case:
                  deposits:
                    add: 100000000000
                    and: 5